# Six-bar mechanism balancing

###### Luis Eduardo Robles Jiménez

With this work, I pretend to find out the behavior of the function with different tools, in order to understand what are the most important variables, find patterns, identify overlapping zones of minima between the functions we're optimizing, graphing data, etc.

In [ ]:
from BetaShF import *
from BetaShM import *
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import scipy as sp
import time


assert ShF([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 1.0
assert ShM([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 1.0

#### Constraints

$$ -0.16m \leq x_{cn}, y_{cn} \leq 0.16m $$

$$ 0.005m \leq t_{cn} \leq 0.04m $$

#### Objective function

In [ ]:
def objective(s, ShF, ShM):
    return 0.5*ShF(s) + 0.5*ShM(s)

## Mathematical analysis 

### Sympy

In [1]:
from sympy import *

#x1c, y1c, t1c, x2c, y2c, t2c, x3c, y3c, t3c, x4c, y4c, t4c, x5c, y5c, t5c = symbols('x1c y1c t1c x2c y2c t2c x3c y3c t3c x4c y4c t4c x5c y5c t5c')
#a = x1c + t1c

In [2]:
#from forceFormula import *
from momentFormula import *

In [3]:
f = Poly(momentF)
count_roots(f)

1

## Sample generation

In [ ]:
nSamples, nVariables = 500, 15
nWeights = nVariables // 3
samples = np.zeros((nSamples, nVariables))
bounds = np.zeros((nVariables, 2))
fitness = np.zeros((nSamples))
shForces = np.zeros((nSamples))
shMoments = np.zeros((nSamples))

In [ ]:
for i in range(nVariables):
    if (i + 1) % 3 == 0: samples[:, i] = np.random.uniform(0.005, 0.04, (nSamples))
    else: samples[:, i] = np.random.uniform(-0.16, 0.16, (nSamples))
for i in range(nSamples):
    fitness[i] = objective(samples[i], ShF, ShM)
"""
for v in range(nVariables):
    bounds[v] = [0.005, 0.04] if (v + 1) % 3 == 0 else [-0.16, 0.16]
worst = 0.0
for s in range(nSamples):
    print("\rIteration:", s, "\tWorst fitness:", worst, end = "")
    p = False
    if s < nSamples // 10: p = True
    r = sp.optimize.differential_evolution(objective, bounds, args = (ShF, ShM), maxiter = 25, popsize = 10, polish = p)
    samples[s] = r.x
    fitness[s] = r.fun
    worst = np.max([worst, fitness[s]])
"""

### Write data to a file

In [ ]:
now = str(int(time.time()))
np.savetxt(now + "Population.txt", samples)
np.savetxt(now + "Fitness.txt", fitness)
np.savetxt(now + "ShForces.txt", shForces)
np.savetxt(now + "ShMoments.txt", shMoments)

### Read data from a file

In [ ]:
"""
# The very best examples, to understand how good solutions behave
fitness = np.loadtxt("1671649703Fitness.txt")
samples = np.loadtxt("1671649703Population.txt")
# Solutions under 1.0
fitness = np.loadtxt("1671919014Fitness.txt")
samples = np.loadtxt("1671919014Population.txt")
"""
# Broader space
fitness = np.loadtxt("1671988185Fitness.txt")
samples = np.loadtxt("1671988185Population.txt")
shForces = np.loadtxt("1671988185ShForces.txt")
shMoments = np.loadtxt("1671988185ShMoments.txt")

print(samples.shape)
print(fitness.shape)
print(shForces.shape)
print(shMoments.shape)

## Dimensionality reduction

### Feature selection

### Feature extraction

#### Find optimal values

In [ ]:
optimal = np.where(fitness < 1)[0]

#### Masses

This is a problem-specific feature that we can get from the variables.

These calculations are intended to be used later on to find out where this is a good representative of the solutions or not.

The mass can be easily calculated with the resulting formula of the following steps.

#### Formula

$$\sigma_i = \frac{m_i}{V_i}$$

$$m_i = V_i \sigma_i$$

$$m_i = (r_i^2 t_i \pi)(\sigma_i)$$

$$m_i = (\sqrt{x_i^2 + y_i^2}^2 t_i \pi)(\sigma_i)$$

$$m_i = (x_i^2 + y_i^2) t_i \pi\sigma_i$$

#### Considering that

$\sigma$ is density. In this problem: $\sigma_i = 7800 \frac{kg}{m^3}$

$m$ is mass $(kg)$

$V$ is volume; of a cylinder in this problem $(m^3)$ 

$x, y$ are the relative coordinates of the weight centers. $(m)$

In [ ]:
masses = np.zeros((nSamples, nWeights))
density = 7800
for i in range(nWeights):
    masses[:, i] = (samples[:, i*3]**2 + samples[:, i*3 + 1]**2) * samples[:, i*3 + 2] * np.pi * density
sumMasses = np.sum(masses, axis = 1)

#### PCA

In [ ]:
modelPCA = PCA()
modelPCA.fit(samples)
dataPCA = modelPCA.transform(samples)
dataPCA.shape

In [ ]:
expVar = modelPCA.explained_variance_ratio_
expVarCum = np.cumsum(expVar)
plt.figure(figsize=(10, 5))
plt.plot(np.concatenate(([0], expVarCum)))
plt.show()

### First two main components

In [ ]:
dataPCA = dataPCA[:, :2]
plt.figure(figsize=(5, 5))
plt.title("PCA first two components")
plt.scatter(dataPCA[:, 0], dataPCA[:, 1], s = 1)
plt.xlabel("PCA 1 -> " + str(np.round(expVar[0]*100, 2)) + "%")
plt.ylabel("PCA 2 -> " + str(np.round(expVar[1]*100, 2)) + "%")
plt.show()

## Data visualization

### Parallel coordinates 

In [ ]:
fig = px.parallel_coordinates(samples, title = "Mechanism balancing samples",
                            color = fitness,
                            labels=["x One", "y One", "t One", "x Two", "y Two", "t Two", "x Three", "y Three", "t Three", "x Four", "y Four", "t Four", "x Five", "y Five", "t Five", "Fitness"],
                            color_continuous_scale=px.colors.sequential.Hot)
fig.show()

In [ ]:
fig = px.parallel_coordinates(masses, title = "Samples' masses",
                            color = fitness,
                            labels=["mass One", "mass Two", "mass Three", "mass Four", "mass Five", "Fitness"],
                            color_continuous_scale=px.colors.sequential.Hot)
fig.show()

In [ ]:
fig = px.parallel_coordinates(np.c_[(shForces[optimal], shMoments[optimal])], title = "Multiobjective optimization behavior",
                            color = fitness[optimal],
                            labels=["Shaking Force", "Shaking Moment", "Fitness"],
                            color_continuous_scale=px.colors.sequential.Hot)
fig.show()

### Violinplot

This graph is created with hopes of seeing if the masses have a tendency. 

Also maybe conclude that the heavier a weight is, the more importance it has.

In [ ]:
fig = plt.figure(figsize =(15, 15))
for i in range(nWeights):
    plt.subplot(2, 3, i + 1)
    plt.title("Weight " + str(i + 1))
    plt.violinplot(masses[:, i])
    plt.violinplot(masses[optimal, i])
plt.subplot(2, 3, 6)
plt.title("Fitness")
plt.violinplot(fitness)
plt.violinplot(fitness[optimal])
plt.show()

### Scatter plot

Let's try to see if there's a relation between the masses of the weights and their fitness.

In [ ]:
fig = plt.figure(figsize =(15, 30))
for i in range(nWeights):
    plt.subplot(5, 2, i*2 + 1)
    plt.title("Weight " + str(i + 1))
    plt.scatter(masses[:, i], fitness, alpha = 0.3, color = '0')
    plt.xlabel("Mass")
    plt.ylabel("Fitness")
    plt.subplot(5, 2, i*2 + 2)
    plt.title("Optimal Weight " + str(i + 1))
    plt.scatter(masses[optimal, i], fitness[optimal], alpha = 0.3, color = '0')
    plt.xlabel("Mass")
    plt.ylabel("Fitness")

In [ ]:
plt.figure(figsize=(15, 10))
plt.grid(which="both")
plt.subplot(1, 2, 1)
plt.scatter(sumMasses, fitness, alpha = 0.3, color = '0')
plt.title("Behavior of total mass")
plt.xlabel("Total mass")
plt.ylabel("Fitness")
plt.subplot(1, 2, 2)
plt.title("Optimal aggregated mass")
plt.ylabel("Mass")
plt.violinplot(sumMasses)
plt.violinplot(sumMasses[optimal])

#### Pareto front

In [ ]:
plt.figure(figsize = (10, 10))
plt.scatter(shForces[optimal], shMoments[optimal], alpha = 0.5, c = fitness[optimal])
plt.xlabel("Shaking Forces")
plt.ylabel("Shaking Moment")

#### Shaking values

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(sumMasses[optimal], shForces[optimal], c = "r", alpha = 0.3)
plt.scatter(sumMasses[optimal], shMoments[optimal], c = "b", alpha = 0.3)